# 6. Interação

_"Um gráfico não é 'desenhado' de uma vez por todas; é 'construído' e reconstruído até que revele todas as relações constituída pela interação dos dados. As melhores operações gráficas são aquelas implementadas pelo próprio tomador de decisão."_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

Visualização fornece meios poderosos para melhor compreensão dos dados. Uma única imagem, porém, tipicamente fornece respostas a, no máximo, algumas poucas perguntas. Através da _interação_ podemos transformar imagens estáticas em ferramentas para exploração: destacando pontos de interessee, dando zoom para revelar padrões finos e interligando através de várias visualizações para entender sobre relações multidimensionais.

No cerne na interação está a noção de uma _seleção_: um meio de indicar para o computador quais elementos ou regiões que estamos interessados. Por exemplo, podemos pairar o mouse sobre um ponto, clicar em várias marcas,  ou desenhar uma moldura em volta de uma região para destacar um pedaço dos dados para análise mais aprofundada.

Junto às codificações visuais e às transformações de dados, o Altair fornece uma abstração de _seleção_ para a criação de interações. Essas seleções englobam três aspectos:

1. Lidar com eventos de entrada para selecionar pontos ou regiões de interesse, como o pairar do mouse, eventos de clicar, de arrastar, de rolar, e de tocar.
2. Generalizar a partir da entrada para construir uma regra de seleção (ou [_predicado_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determina se um registro de um dado fornecido está na seleção ou não.
3. Usar o predicado de seleção para configurar dinamicamente uma visualização através de _codificações condicionais_, _transformações de filtro_ ou _domínios de escala_.

Esse capítulo introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, como consultas dinâmicas, panorama &amp; zoom, detalhes em demanda, e interligar &amp; varrer.  

_Esse capítulo é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import pandas as pd
import altair as alt

## 6.1. Datasets

Iremos visualizar uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- Um dataset `cars` de carros dos anos 70 e começo dos anos 80,
- Um dataset `movies` de filmes, anteriormente usados no notebook [Transformação de Dados](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- Um dataset contendo dez anos de [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`) preços de ações,
- Um dataset `stocks` das ações de empresas de tecnologia e
- Um dataset `flights` dos voos de avião, incluindo hora de decolagem, distância e atraso de pouso

In [ ]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.2. Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando num ponto para destacá-lo. Usando o dataset `cars`, vamos começar com um gráfico de dispersão da potência (em cavalos) por milhas por galão, com uma codificação de cor para o número de cilindros no motor do carro.

Além disso, vamos criar uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida por um _único valor_. Por padrão, a seleção usa um clique de mouse para determinar o valor selecionado. Para registrar uma seleção com um gráfico, devemos adicionar usando o método `.add_selection()`.

Uma vez que nossa seleção tenha sido definida, podemos usá-la como parâmetro para _codificações condicionais_, que aplicam uma codificação diferente dependendo se o registro do dado está dentro ou fora da seleção. Por exemplo, considere o seguinte código:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Essa definição de codificação afirma que os pontos do dado contidos em `selection` devem ser coloridos de acordo com o campo `Cylinder`, enquanto pontos do dado não selecionados devem usar um `grey` (_cinza_) padrão. Uma seleção vazia inclui _todos_ os pontos do dado e, então, inicialmente todos os pontos serão coloridos.

_Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado da seleção e retornar a uma seleção "vazia".)_

In [ ]:
selection = alt.selection_single()
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x= alt.X('Horsepower:Q', title = 'Potência (em cavalos)'),
    y= alt.Y('Miles_per_Gallon:Q', title = 'Milhas por galão'),
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

Claro, destacar pontos do dado individuais um a um não é particularmente divertido! Como veremos, seleções de valor único fornecem um alicerce útil para construir interações mais poderosas. Ademais, seleções de valor único são apenas uma dos três tipos de seleção fornecidas pelo Altair:

- `selection_single` - selecione um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - selecione vários valores discretos. O primeiro valor é selecionado clicando com o mouse e valores adicionais são inseridos usando Shift+Click. 
- `selection_interval` - selecione uma faixa contínua de valores, iniciados arrastando o mouse enquanto clicado.

Vamos comparar cada uma desses tipos de seleção lado a lado. Para manter nosso código arrumado, vamos primeiro definir uma função (`plot`) que gera especificações dos gráficos de dispersão como o acima. Podemos passar uma seleção para a função `plot` para aplicá-lo ao gráfico:

In [ ]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x= alt.X('Horsepower:Q', title = 'Potência (em cavalos)'),
        y= alt.Y('Miles_per_Gallon:Q', title = 'Milhas por galão'),
        color=alt.condition(selection, alt.Color('Cylinders:O', title = 'Cilindros'), alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar três variantes do gráfico, um para cada tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta a interação Shift+Click para inserir vários pontos na seleção. O terceiro gráfico (`interval`) gera uma região selecionada (ou _brush_) a partir do arrasto do mouse clicado. Uma vez criada, você pode mover a área varrida pelo gráfico para selecionar diferentes pontos, ou rolar quando o cursor está dentro da área varrida para aumentar ou diminuir o tamanho da área.

_Tente interagir com os gráficos abaixo!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

Os exemplos acima usam interações padrão (Clicar, Shift+Click, Arrastar) para cada tipo de seleção. Podemos costumizar mais as interações fornecendo especificações do evento de entrada usando a [sintaxe de seleção de evento do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para acionar com eventos de `pairar do mouse` ao invés de eventos de `click`

_Segure a tecla Shift no segundo gráfico para "pintar" com os dados!_

In [ ]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Pairar do Mouse)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Pairar do Mouse)')
)

Agora que cobrimos o básico das seleções do Altair, vamos fazer um tour pelas diversas técnicas de interação que elas permitem!

## 6.3. Consultas Dinâmicas

_Consultas dinâmicas_ habilitam uma exploração reversível dos dados rápida para isolar padrões de interesse. Como definido por [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- fornece limites visuais para a faixa da consulta,
- fornece uma representação gráfica dos dados e o resultado da consulta,
- dá um retorno imediato do resultado após cada ajuste da consulta,
- e possibilita usuários novatos começar a trabalhar com pouco treino.

Uma abordagem comum é manipular os parâmetros da consulta usando ferramentas da interface do usuário padrão como uma barra deslizante, botões de rádio, e menus drop-down. Para gerar ferramentas de consulta dinâmica, podemos aplicar a operação `bind` da seleção para um ou mais campos de dados que queremos consultar.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de avaliações de filmes (do Rotten Tomatoes e IMDB), podemos adicionar uma seleção sobre o campo `Major_Genre` para habilitar filtragens por gênero interativas.




Para começar, vamos extrair os gêneros (não nulos) dos dataset `movies`.

In [ ]:
df = pd.read_json(movies) # carregar dados de filmes
genres = df['Major_Genre'].unique() # obter valores de campo exclusivos
genres = list(filter(lambda d: d is not None, genres)) # filtrar valores
genres.sort() # classificar em ordem alfabética

Para uso futuro, vamos definir também uma lista de valores `MPAA_Rating` sem repetição:

In [ ]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar uma seleção `single` ligada a um menu drop-down.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # nomeie a seleção como 'Selection'
    fields=['Major_Genre'], # limitar a seleção ao campo Major_Genre
    init={'Major_Genre': genres[0]}, # use a primeira entrada do gênero como valor inicial
    bind=alt.binding_select(options=genres) # vincular-se a um menu de valores de gênero exclusivos
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

Nossa construção acima enaltece vários aspectos de seleções:

- Damos à seleção o nome (`'Select'`). Esse nome não é necessário, mas nos permite influenciar o rótulo de texto do menu de consulta dinâmica gerado. (_O que acontece se você remove o nome? Experimente!_)
- Restringimos a seleção a um campo de dado específico (`Major_Genre`). Anteriormente, quando usamos uma seleção `single`, a seleção era mapeada para pontos de dados individuais. Limitando a seleção para um campo específico podemos selecionar _todos_ os pontos do dado cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Inicializamos `init=...` a seleção para um valor inicial.
- `Vinculamos` a seleção para uma ferramenta da interface, nesse caso um menu drop-down via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar o canal de opacidade.

### 6.3.1. Vincular Seleções para Várias Entradas

Uma instância de seleção pode ser vinculada a _várias_ ferramentas de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para _ambos_ `Major_Genre` e `MPAA_Rating`, usando botões de rádio ao invés de um menu. Nossa seleção `single` é definida agora sobre um único _par_ de gêneros e valores de classificação MPAA

_Procure por conjunções de gênero e classificação. Há algum filme de terror com classificação G ou PG?_

In [ ]:
# seleção de valor único em pares [Major_Genre, MPAA_Rating]
# use valores específicos conectados como os valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# gráfico de dispersão, modifique a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

_Fatos interessantes: A classificação PG-13 não existia quando os filmes [Jaws](https://www.imdb.com/title/tt0073195/)  e [Jaws 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber uma classificação PG-13 foi [Red Dawn](https://www.imdb.com/title/tt0087985/) de 1984._

### 6.3.2. Usando Visualizações como Consultas Dinâmicas

Embora ferramentas padrão de interface mostrem os valores dos parâmetros de consulta _possíveis_, elas não exibem a _distribuição_ desses valores. Podemos também querer usar interações mais ricas, como seleções `multi-value` e `interval`, ao invés de ferramentas de entrada que selecionam apenas um valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais tanto para visualizar dados e suportar consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção `interval` para destacar dinamicamente filmes de períodos de tempo selecionados.

*Interaja com o histograma anual para explorar filmes de épocas diferentes. Viu alguma pista de [amostra enviesada](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como ano e avaliações da crítica se relacionam?)*

_Os anos vão de 1930 a 2040! Os filmes do futuro estão sendo produzidos ou há um erro de um século? Além disso, dependendo de qual fuso horário você está, talvez você veja uma pequena protuberância em 1969 ou 1970. Porque será? (Veja o final do notebook para uma explicação!)_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limitar a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# gráfico de dispersão, modifique a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

O exemplo acima fornece consultas dinâmicas usando uma _seleção interligada_ entre gráficos:

- Criamos uma seleção `interval` (`brush`) e colocamos `encodings=['x']` para limitar a seleção apenas para o eixo x, resultando numa seleção de intervalo unidimensional.
- Registramos `brush` com o nosso histograma de filme por ano via `.add_selection(brush)`.
- Usamos `brush` em uma codificação condicional para ajustar a `opacity` dos gráficos de dispersão.

Essa técnica de interação de selecionar elementos em um gráfico e observar destaques interligados em um ou mais gŕaficos é conhecida como [_varrer &amp; interligar_](https://en.wikipedia.org/wiki/Brushing_and_linking).

## 6.4. Deslocar &amp; Ampliar

O gráfico de dispersão da classificação dos filmes está bem desorganizado, dificultando a análise dos pontos em regiões mais densas. Usando as técnicas de interação de _deslocar_ e _ampliar_, podemos inspecionar regiões densas mais de perto.

Vamos começar pensando sobre como podemos expressar deslocamento e ampliação usando seleções do Altair. O que define o "viewport (janela de visualização)" de um gráfico? _Domínios de escala do eixo!_

Podemos mudar o domínio de escala para modificar a faixa de valores dos dados que estão sendo visualizados. Para fazer isso interativamente, podemos vincular uma seleção `interval` para domínios de escala com o código `bind='scales'`. O resultado é que em vez de uma ferramenta que permite selecionar e ampliar um intervalo específico, agora podemos arrastar e ampliar toda a área do gráfico!

_No gráfico abaixo, clique e arraste para deslocar a vista ou role para dar ampliar (escala) na vista. O que você pode descobrir sobre a precisão dos valores de classificações fornecidas?_

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Ampliando, podemos ver que os valores de classificação tem precisão limitada! As classificações do Rotten Tomatoes são inteiras, enquanto as classificações do IMDB estão truncadas por dezenas. Como resultado, há uma sobre-exibição mesmo quando ampliamos, com vários filmes compartilhando os mesmos valores de classificação._

Lendo o código acima, talvez você note o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço é reservada para marcações e rótulos axiais. Por que fazer isso? Quando deslocamos e ampliamos, os rótulos axiais podem mudar e causar uma deslocação do título do eixo. Colocando uma extensão mínima podemos reduzir movimentos distratores na exibição._Tente mudar o valor de `minExtent`, por exemplo colocando-o no zero e então dimunua o zoom para ver o que acontece quando rótulos de eixo maiores entram na visualização._

O Altair também inclui um atalho para adicionar a um gráfico deslocamento e ampliação. Ao invés de diretamente criar uma seleção, você pode chamar `.interactive()` para fazer o Altair gerar automaticamente uma seleção vinculada às escalas do gráfico:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

Por padrão, vínculos de escala para seleções incluem tanto o canal de codificação `x` quanto o canal de codificação `y`. E se queremos limitar a interação de deslocar e ampliar apenas para uma dimensão? Podemos chamar `encodings=['x']` para restringir a seleção apenas para o canal `x`:

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use extensão mínima para estabilizar o posicionamento do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

_Quando ampliamos ao longo de um único eixo, o formato dos dados visualizados pode mudar, potencialmente afetando nossa percepção das relações nos dados.  [Escolhendo uma razão de aspecto ideal](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante para o design da visualização!_

## 6.5. Navegação: Panorama + Detalhe

Quando deslocamos e ampliamos, ajustamos diretamente a janela de visualização de um gráfico. A estratégia de navegação relacionada de _Panorama + Detalhe_ usa, porém, uma exibição geral para mostrar _todos_ os dados, enquanto suporta seleções que deslocam e ampliam numa exibição de um foco separado.

Abaixo temos dois gráficos de área mostrando uma década das flutuações dos preços do índice da ação S&amp;P 500. Inicialmente ambos os gráficos mostram a mesma faixa de dados. _Clique e arraste no gráfico resumido embaixo para atualizar o foco da exibição e examinar períodos de tempo específicos._

In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)
  
alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

Em contraste com o caso anterior deslocar &amp; zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Na verdade, queremos vincular a seleção ao domínio de escala de um _outro_ gráfico. Para fazer isso, atualizamos o canal de codificação `x` para nosso gráfico principal, colocando a propriedade `domain` da escala para referenciar nossa seleção `brush`. Se nenhum intervalo for definido (a seleção está vazia), o Altair ignora a varredura e usa os dados ocultos para determinar o domínio. Quando um intervalo varrido é criado, o Altair usa-o como o `domain` da escala para o gráfico principal.

## 6.6. Detalhes sob Demanda

Assim que encontramos pontos de interesse dentro da visualização, geralmente queremos saber mais sobre eles. _Detalhe sob demanda_ se refere consultar interativamente buscando mais informações sobre valores selecionados. `Tooltips` são um recurso útil para fornecer detalhes sob demanda. Porém, `tooltips` tipicamente mostram apenas informações para um ponto do dado por vez. Como podemos mostrar mais?

O gráfico de dispersão das classificações dos filmes inclui valores discrepantes (outliers) potencialmente interessantes onde o IMDB e o Rotten Tomatoes discordam. Vamos criar um plot que nos permite selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro através da interação de pairar ou da interação de clicar, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

_Passe o mouse sobre pontos no gráfico de dispersão abaixo para ver um rótulo de destaque e título. Use Shift+Click nos pontos para fazer anotações persistentes e ver vários rótulos ao mesmo tempo. Quais filmes são amados pela crítica do Rotten Tomatoes, mas não pelo público geral do IMDB (e vice versa)? Veja se você consegue achar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!_

In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecione ao passar o mouse
    nearest=True,    # selecione o ponto mais próximo do cursor do mouse
    empty='none'     # a seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia não corresponde a nenhum ponto
)

# codificações de gráficos de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)
  
# base compartilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# pontos do gráfico de dispersão de camadas, anotações de halo e rótulos de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

O exemplo acima adiciona três novas camadas para o gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto em preto mostrando o nome do filme. Ademais, esse exemplo usa duas seleções em conjunto:

1. Uma seleção única (`hover`) que inclui `nearest=True` para automaticamente selecionar o ponto do dado que está mais próximo do cursor.
2. Uma seleção multi (`click`) para criar seleções persistentes via Shift+Click.

Ambas as seleções incluem o `empty='none'` para indicar que nenhum ponto deve ser incluso se a seleção está vazia. Essas seleções são então combinadas num único predicado de filtro &mdash; o _ou_ lógico de `hover` e `click` &mdash; para incluir pontos que estão em _qualquer_ uma das duas seleções. Usamos esse predicado para filtar as novas camadas para mostrar anotações e rótulos apenas para pontos selecionados.

Usando seleções e camadas, podemos fazer vários designs diferentes sob demanda! Por exemplo, aqui está uma série temporal de preços de ações de tecnologia em escala logarítmica, anotadas com uma diretriz e rótulos para a data mais próxima ao cursor:

In [ ]:
# selecione um ponto para o qual fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limitar a seleção ao valor do eixo x
    on='mouseover',  # selecione em eventos de mouseover
    nearest=True,    # selecione o ponto de dados mais próximo do cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# definir nosso gráfico de linha base de preços de ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, #gráfico de linha básica
    
    # adicione uma marca de regra para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # adicionar marcas circulares para pontos de tempo selecionados, ocultar pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adicione texto com traços brancos para fornecer um fundo legível para rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adicione rótulos de texto para preços de ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

_Colocando o que aprendemos até agora em prática: você consegue mudar o gráfico de dispersão acima (aquele com a consulta dinâmica nos anos) para incluir uma marca de regra (`mark_rule`) que mostra a média das classificações IMDB (ou Rotten Tomatoes) para os dados contidos na seleção `interval` dos anos?_

## 6.7. Varrer &amp; Interligar, Revisitado

Anteriormente nesse capítulo vimos um exemplo de _varrer &amp; interligar_: usando um histograma de consulta dinâmica para destacar pontos num gráfico de dispersão de filmes. Aqui visitaremos alguns exemplos adicionais envolvendo seleções interligadas.

Voltando ao dataset `cars`, podemos usar o operador `repeat` para construir uma [Matriz de gráficos de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência em cavalos. Podemos definir uma seleção `interval` e inclui-la _dentro_ da nossa especificação repetida de gráficos de dispersão para habilitar seleções ligadas dentre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para varrer &amp; interligar!_

In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolver todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

Note acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que dentre todos os gráficos apenas uma varredura pode estar ativa por vez. Entretanto, em alguns casos nos podemos querer definir varreduras em vários gráficos e juntar os resultados. Se usarmos `resolve='union'`, a seleção será a _união_ de todas as varreduras: se um ponto está dentro de qualquer área varrida será selecionado. Em contraste com o uso de `resolve='intersect'`, a seleção consistirá na _interseção_ de todas as varreduras: apenas pontos que estão dentro de todas as áreas varridas serão selecionados.

_Tente colocar o parâmetro `resolve` para `'union'` e para `'intersect'` e veja como se altera a lógica de seleção._

### 6.7.1. Filtragem Cruzada

Os exemplos sobre varre &amp; interligar que vimos usam codificações condicionais, por exemplo para mudar os valores da opacidade em resposta a uma seleção. Outra opção é usar a seleção definida em um gráfico para _filtrar_ o conteúdo de outro gráfico.

Vamos construir uma coleção de histogramas para o dataset `flights`: atraso `delay` (o quão cedo ou o quão tarde o voo pousou, em minutos), distância percorrida (em milhas) `distance` e horário de decolagem `time` (horário do dia). Usaremos o operador `repeat` para criar os histogramas e adicionar uma seleção `interval` para o eixo `x` com varreduras que funcionam por interseção.

Em particular, cada histograma consistirá de duas camadas: uma camada de fundo cinza e uma camada superior azul, com a camada superior filtrada pela nossa seleção de varreduras por interseção. O resultado é uma interação de _filtragem cruzada_ com os três gráficos!

_Use as varreduras arrastando nos gráficos abaixo. Ao selecionar voos com atrasos maiores ou menores, como a distribuição da distância percorrida e do horário de decolagem se comportam?_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 caixas
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # sem título no eixo y
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # atrasos de fixação > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionárias
).configure_view(
    stroke='transparent' # sem esboço
)

_Usando a filtragem cruzada você pode observar que voos atrasados tem mais chance de terem decolado mais tarde no dia. Esse fenômeno é comum para viajantes frequentes: o atraso pode se propagar ao longo do dia, afetando voos subsequentes daquele mesmo avião. Para chegar na hora com chances melhores, voe cedo!_

A combinação de vários gráficos e seleções interativas pode possibilitar formas valiosas de entendimento multidimensional, tornando até histogramas básicos em aparatos de entrada poderosos para fazer perguntas sobre um dataset!

## 6.8. Resumo 

Para mais informações sobre as opções de interações suportadas pelo Altair, por favor, consulte a [Documentação de seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre costumizar gestores de eventos, por exemplo para compor várias técnicas de interação ou suportar entradas baseadas no toque em dispositivos móveis, veja a [Documentação de seleção do Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Quer saber mais?
- A abstração _seleção_ foi introduzida no artigo [Vega-Lite: Uma gramática de Gráficos Interativos](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- O sistema PRIM-9 (para projeções, rotações, isolamento e disfarce em até 9 dimensões) é um das ferramentas de viusalização interativa mais recentes, feita no começo dos anos 70 por Fisherkeller, Tukey, &amp; Friedman. [A retro demo video survives!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
- O conceito de varrer &amp; interligar foi aperfeiçoado por Becker, Cleveland, &amp; Wilks em seu artigo de 1987 [Gráficos Dinâmicos para Análise de Dados](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para um resumo esclarecedor de técnicas de interação para visualização, veja [Dinâmica Interativa para Análise Visual](https://queue.acm.org/detail.cfm?id=2146416) de Heer &amp; Shneiderman.

### 6.8.1. Apêndice: Na Representação do Tempo

Lá atrás observamos que uma pequena protuberância no número de filmes em 1969 ou 1970. De onde vem essa protuberância? e por que 1969 _ou_ 1970? A resposta vem de uma combinação de dados faltantes e como nosso computador representa o tempo.

Internamente, datas e horários são representadas em relação à [Era Unix](https://en.wikipedia.org/wiki/Unix_time), em que o tempo "zero" corresponde à meia noite do dia 1 de Janeiro de 1970 no [fuso UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que muda junto ao [Meriano de Greenwich](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes que estão com a data de lançamento faltando (`null`). Esses valores (`null`) são interpretados como tempo `0` e então mapeiam para 1 de Janeiro de 1970 no fuso UTC. Se você mora nas Américas &ndash; e, portanto, em fusos mais "cedo" &ndash; esse ponto no tempo corresponde a uma hora mais cedo no dia 31 de dezembro de 1969 no seu fuso horário. Por outro lado, se você vive próximo ou ao leste do meridiano de Greenwich, a data no seu fuso horário será dia 1 de Janeiro de 1970.

A moral? Sempre seja cético com seus dados e seja atencioso, afinal, como os dados são representados (sejam datas, números em ponto flutuante ou latitudes e longitudes, _etc._) podem, as vezes, nos levar a artefatos que impactam a análise! 